In [4]:
import requests

# Base URL for Solana RPC endpoint
rpc_url = "https://api.mainnet-beta.solana.com"

# List of wallet addresses holding some tokens (example)
wallet_addresses = [
    'C4su4UzsPgbDApTEWFThKwbTyHE9ZoRnV7TXwrtL55SB',
    'AnsWpQ5KzJpns4fTXQWzWg8YakJMrDS2T6M3TpUX4t5U',
    '93AX7iJUTbzmzB4vHXCRHjfLyrskAfNd1ReRQkfRUnXL',
    # Add more wallet addresses...
]

def get_token_accounts_by_owner(wallet_address):
    headers = {'Content-Type': 'application/json'}
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getTokenAccountsByOwner",
        "params": [
            wallet_address,
            {
                "programId": "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA",
                "encoding": "jsonParsed"
            }
        ]
    }

    response = requests.post(rpc_url, json=payload, headers=headers)
    return response.json()

# Loop through each wallet and retrieve their token accounts
wallet_to_mints = {}

for wallet in wallet_addresses:
    accounts = get_token_accounts_by_owner(wallet)
    # Extract mint addresses for each wallet
    mints = [account['account']['data']['parsed']['info']['mint'] for account in accounts.get('result', {}).get('value', [])]
    
    # Store the mint addresses for each wallet
    wallet_to_mints[wallet] = mints

# Print the mint addresses associated with each wallet
print(wallet_to_mints)

{'C4su4UzsPgbDApTEWFThKwbTyHE9ZoRnV7TXwrtL55SB': [], 'AnsWpQ5KzJpns4fTXQWzWg8YakJMrDS2T6M3TpUX4t5U': [], '93AX7iJUTbzmzB4vHXCRHjfLyrskAfNd1ReRQkfRUnXL': []}


In [ ]:
import asyncio
import nest_asyncio
from solana.rpc.async_api import AsyncClient
from solana.rpc.api import Pubkey

nest_asyncio.apply()

async def get_wallet_transactions(wallet_address, limit=10):
    pubkey = Pubkey.from_string(wallet_address)
    async with AsyncClient("https://api.mainnet-beta.solana.com") as client:
        # Fetch the signatures for the wallet address
        signatures_response = await client.get_signatures_for_address(pubkey, limit=limit)

        # Check if the response contains 'value' and that it's a list
        if not hasattr(signatures_response, 'value') or not isinstance(signatures_response.value, list):
            return []  # Return an empty list if no 'value' or incorrect format

        # Extract signatures from the response
        transaction_signatures = [sig.signature for sig in signatures_response.value]

        # Fetch the transaction details using get_transaction (instead of get_parsed_transactions)
        transactions = []
        for signature in transaction_signatures:
            transaction = await client.get_transaction(signature, "jsonParsed", max_supported_transaction_version=0)
            transactions.append(transaction)

        return transactions

async def main():
    wallet_address = "BHcTB516BrHzqDyFjMnDfWhSdLLxKKbihf4RauCJtD1h"
    transactions = await get_wallet_transactions(wallet_address)
    for tx in transactions:
        await asyncio.sleep(0.5)
        print(f"Signature: {tx.transaction.signatures[0]}")
        print(f"Timestamp: {tx.block_time}")
        print("Instructions:")
        for instruction in tx.transaction.message.instructions:
            print(f"  Program: {instruction.program_id}")
        print("-" * 20)

asyncio.run(main())


In [27]:
import csv
import requests
import os
from pathlib import Path
from dotenv import load_dotenv
from pprint import pprint

# Load environment variables
load_dotenv()

# RPC Endpoint
RPC_ENDPOINT = os.getenv("RPC_ENDPOINT")

# Fetch token holders using getTokenLargestAccounts
def get_token_holders(token_address):
    payload = {
        "jsonrpc": "2.0",
        "id": "getTokenLargestAccounts",
        "method": "getTokenLargestAccounts",
        "params": [token_address],
    }
    response = requests.post(RPC_ENDPOINT, json=payload)
    if response.status_code == 200:
        data = response.json().get("result", {}).get("value", [])
        pprint(data)
        return [(holder["address"], float(holder["amount"])) for holder in data]
    return []
    
tokens = ["BbzVAbu8rrmEGQRSFhS2tF42Agp1FsBtcEMoYLH3r84J",
"BBBmxremG7CnQCg1BpArvEY5QPH1J7fydn68fNKMcZne",
"7zaAmppPP7tstHMBqvrsy5J11ZGS2BkGbHv2xey5iNBo",
"CtQ7hnCGqLmFeW87tom2fHGTsCraTjzogjzALQLykXqL",
"DLktwpXUh9ydTGxcZRvkESJFggLiezu2VFP72dapump",
"6cfZ9tJhfsR7bZfVWAPtu2UrPqfWomud67Q2RC5Ypump",
"FnJoStz95yP5B9kSg2BiCE8FdsiL4S2AnZLrtMJYBdHg",
"9NcdSJeMvgMYpFgKJGrJyBqMTkWDApzZv7kwFiFpnWMy",
"4Sk8TVAoWaCPET62MRGsuKi8e2VwyAgtbiHS1MxP7vcn",
"FBmV4Zp3YiXSaybUKV6Dnh5W489cmQCRbnBiUnEwmoon"
]
for token in tokens:
    holders = get_token_holders(token)
    print(f"Token: {token}")
    print("Holders:")
    for holder, amount in holders:
        print(f"  {holder}: {amount}")
    print("\n")


[{'address': 'CFNS3HNTeTcM7hgd9kuPXEyXDAdw57he7WEhZz8jNSkd',
  'amount': '937416692251523',
  'decimals': 6,
  'uiAmount': 937416692.251523,
  'uiAmountString': '937416692.251523'},
 {'address': 'kUAdcZnmfUhvMi93V9U12Ct3J7FzmzLJuhm1bH73Ggw',
  'amount': '59421073670844',
  'decimals': 6,
  'uiAmount': 59421073.670844,
  'uiAmountString': '59421073.670844'},
 {'address': '9AAevGavE8gtNFHNxUYsqaCUiH2adcuF5sPkQztXJPKr',
  'amount': '2209927414907',
  'decimals': 6,
  'uiAmount': 2209927.414907,
  'uiAmountString': '2209927.414907'},
 {'address': 'B85h5oLEKFajVc5ScfDG28bjVX8jFbPPyRwTgyc5mHP6',
  'amount': '199859425503',
  'decimals': 6,
  'uiAmount': 199859.425503,
  'uiAmountString': '199859.425503'},
 {'address': 'DWMDrnxJ2gZmNY56RR3rnS8nccZRvBSr2374LDbGbWpi',
  'amount': '160660302136',
  'decimals': 6,
  'uiAmount': 160660.302136,
  'uiAmountString': '160660.302136'},
 {'address': '13CqyALkW64DCHaYiajn8FZy95TCjt9nrh2HhjCxfwWb',
  'amount': '104427188314',
  'decimals': 6,
  'uiAmount'

[{'address': 'AesXonmNsVKNqKkSV1gPKRGm2rvoLq5PZiuQVqv8Akn2',
  'amount': '37826842049989843',
  'decimals': 6,
  'uiAmount': 37826842049.98984,
  'uiAmountString': '37826842049.989843'},
 {'address': '3HhQfdF9gByPdyiZ3u12j619uNHm8jNxE5cjmL1X4PtL',
  'amount': '11793343801685016',
  'decimals': 6,
  'uiAmount': 11793343801.685017,
  'uiAmountString': '11793343801.685016'},
 {'address': 'FyDFjjxHH7spfVHnXJAfKE3WAkz1dmvqHo9houC3o6Yo',
  'amount': '8381935202348765',
  'decimals': 6,
  'uiAmount': 8381935202.348765,
  'uiAmountString': '8381935202.348765'},
 {'address': '5oEfT4HAP5yhKSA83kwnpxSbxpo1SnGoUb2i7AVvQHnR',
  'amount': '5154220880233098',
  'decimals': 6,
  'uiAmount': 5154220880.233098,
  'uiAmountString': '5154220880.233098'},
 {'address': '8fTVMJ18akHJ49kTydUP1Js4j849DP3ZpA9rB8RfdHcF',
  'amount': '4259319040924150',
  'decimals': 6,
  'uiAmount': 4259319040.92415,
  'uiAmountString': '4259319040.92415'},
 {'address': '2cFAtpWbnDTCmq5RkCFjEs9sHiGajFL56bbJfmhDRkKY',
  'amount':

In [ ]:
import requests

RPC_ENDPOINT = "https://api.mainnet-beta.solana.com"

def get_token_accounts_by_owner(wallet_address):
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getTokenAccountsByOwner",
        "params": [
            wallet_address,
            {"programId": "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA"},
            {"encoding": "jsonParsed"}
        ]
    }
    
    try:
        response = requests.post(RPC_ENDPOINT, json=payload, timeout=10)
        response.raise_for_status()
        data = response.json().get("result", {}).get("value", [])

        # Extract mints
        mints = [account["account"]["data"]["parsed"]["info"]["mint"] for account in data]
        return mints

    except requests.exceptions.RequestException as e:
        print(f"Error fetching token accounts: {e}")
        return []

# Example usage
wallet_address = "EA2giVUJRAmxL4K9NHydnDsXTNqtFxXMfW5mzSWVANrh"
tokens = get_token_accounts_by_owner(wallet_address)
print(tokens)


In [4]:
import os
import requests
import time
from typing import Union, List, Optional
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class RunRPCMethod:
    BASE_URL = "https://mainnet.helius-rpc.com"

    def __init__(self, max_retries: int = 3, initial_delay: int = 1):
        self.api_key = os.getenv("HELIUS_API_KEY")
        if not self.api_key:
            raise ValueError("HELIUS_API_KEY not found in .env file")

        self.max_retries = max_retries
        self.initial_delay = initial_delay

    @property
    def rpc_url(self) -> str:
        return f"{self.BASE_URL}/?api-key={self.api_key}"

    def _make_api_request(self, payload: dict):
        """Make API requests with exponential backoff."""
        delay = self.initial_delay
        for attempt in range(self.max_retries):
            try:
                response = requests.post(self.rpc_url, json=payload, headers={"Content-Type": "application/json"}, timeout=10)
                response.raise_for_status()
                return response.json()
            except requests.exceptions.RequestException as e:
                if attempt == self.max_retries - 1:
                    print(f"Max retries reached. Failed request: {payload}")
                    raise e
                print(f"Attempt {attempt + 1} failed. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # Exponential backoff

    def call(self, method: str, request_id: Union[int, str], params: Optional[List] = None):
        """Generic method to call any Solana RPC method with retries."""
        payload = {
            "jsonrpc": "2.0",  # Hardcoded constant
            "id": request_id,
            "method": method,
            "params": params or []
        }
        return self._make_api_request(payload)

    # Predefined methods for common RPC calls
    def get_account_info(self, request_id: Union[int, str], account: str, encoding: str = "jsonParsed"):
        """Fetch account info from Solana blockchain."""
        return self.call("getAccountInfo", request_id, [{"encoding": encoding, "account": account}])

    def get_block_height(self, request_id: Union[int, str]):
        """Fetch the current block height."""
        return self.call("getBlockHeight", request_id)

    def get_recent_blockhash(self, request_id: Union[int, str]):
        """Fetch the most recent blockhash."""
        return self.call("getRecentBlockhash", request_id)

    def get_transaction(self, request_id: Union[int, str], signature: str, encoding: str = "json"):
        """Fetch transaction details by signature."""
        return self.call("getTransaction", request_id, [signature, {"encoding": encoding}])


In [2]:
# rom rpc_client import RunRPCMethod

# Initialize the RPC client
rpc = RunRPCMethod()

# Make calls using predefined methods
try:
    account_info = rpc.get_account_info(request_id="account_info", account="2aoZTcWvDZKeyufWkYbJ9mYaoSD1P9HPKzEcZ79GZHbx")

    block_height = rpc.get_block_height(request_id="block_1")
    print("Block Height:", block_height)

    recent_blockhash = rpc.get_recent_blockhash(request_id="blockhash_abc")
    print("Recent Blockhash:", recent_blockhash)

    transaction_info = rpc.get_transaction(request_id="txn_456", signature="QBcTdN1VuSVffWzPXpnzN6t8UZXZe3Gm16pAaTuhSQkXfeDvWzVhhzZWevuhA7pNb4Cgmm6kLjBUAwiX7YAQcAC")
    print("Transaction Info:", transaction_info)

except Exception as e:
    print("Error:", e)


Block Height: {'jsonrpc': '2.0', 'result': 300370303, 'id': 'block_1'}
Attempt 1 failed. Retrying in 1 seconds...
Attempt 2 failed. Retrying in 2 seconds...
Max retries reached. Failed request: {'jsonrpc': '2.0', 'id': 'blockhash_abc', 'method': 'getRecentBlockhash', 'params': []}
Error: 410 Client Error: Gone for url: https://mainnet.helius-rpc.com/?api-key=cde2166e-a9cc-4f20-aab6-931319852b4a


In [9]:
import websockets
import asyncio
import nest_asyncio
import json

nest_asyncio.apply()

HELIUS_WS_URL = "wss://mainnet.helius-rpc.com/?api-key=cde2166e-a9cc-4f20-aab6-931319852b4a"

async def listen_helius():
    async with websockets.connect(HELIUS_WS_URL) as ws:
        mint_subscription = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "accountSubscribe",
            "params": ["TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA", {"encoding": "jsonParsed"}]
        }
        raydium_subscription = {
            "jsonrpc": "2.0",
            "id": 2,
            "method": "programSubscribe",
            "params": ["675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8", {"encoding": "jsonParsed"}]
        }

        await ws.send(json.dumps(mint_subscription))
        await ws.send(json.dumps(raydium_subscription))
        print("Listening for token mints & Raydium pool creations with enhanced anti-Rug and frontrun checks...")

        while True:
            try:
                message = await ws.recv()
                print(message)
            except Exception as e:
                print(e)

if __name__ == "__main__":
    import asyncio
    asyncio.run(listen_helius())

Listening for token mints & Raydium pool creations with enhanced anti-Rug and frontrun checks...
{"jsonrpc":"2.0","result":869444,"id":1}
{"jsonrpc":"2.0","error":{"code":-32602,"message":"Invalid Request: Invalid pubkey provided"},"id":2}
{"jsonrpc":"2.0","method":"accountNotification","params":{"result":{"context":{"slot":322915074},"value":{"lamports":934087680,"data":["...","base64"],"owner":"BPFLoader2111111111111111111111111111111111","executable":true,"rentEpoch":18446744073709551615,"space":134080}},"subscription":869444}}


{"jsonrpc":"2.0","method":"accountNotification","params":{"result":{"context":{"slot":322915076},"value":{"lamports":934087680,"data":["...","base64"],"owner":"BPFLoader2111111111111111111111111111111111","executable":true,"rentEpoch":18446744073709551615,"space":134080}},"subscription":869444}}
{"jsonrpc":"2.0","method":"accountNotification","params":{"result":{"context":{"slot":322915077},"value":{"lamports":934087680,"data":["...","base64"],"owner":"BPFLoader2111111111111111111111111111111111","executable":true,"rentEpoch":18446744073709551615,"space":134080}},"subscription":869444}}
{"jsonrpc":"2.0","method":"accountNotification","params":{"result":{"context":{"slot":322915078},"value":{"lamports":934087680,"data":["...","base64"],"owner":"BPFLoader2111111111111111111111111111111111","executable":true,"rentEpoch":18446744073709551615,"space":134080}},"subscription":869444}}
{"jsonrpc":"2.0","method":"accountNotification","params":{"result":{"context":{"slot":322915078},"value":{"la

KeyboardInterrupt: 